<a href="https://colab.research.google.com/github/CloneIsa/Max6675-GoogleColab/blob/main/Max6675%26MQTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando a SDK do Google


In [ ]:
!pip install -q -U google-generativeai

Configurando a minha API Key

In [ ]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

Listar os modelos disponíveis

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Configurando os parâmetros de temperatura (Criatividad da IA)

In [ ]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

Configurações de segurança

In [ ]:
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE"
}

Inicializando o modelo - após digitar model e colocar o ponto final, clique Ctrl e espaço para visualizar a listagem

In [ ]:
model = genai.GenerativeModel(
    model_name = 'gemini-1.0-pro',
    generation_config = generation_config,
    safety_settings=safety_settings)

Validando a saída:

In [ ]:
response = model.generate_content("Vamos aprender conteúdos sobre IA. Me dê sugetões")
print(response.text)

**Conceitos Fundamentais:**

* Definição de Inteligência Artificial (IA)
* Tipos de IA: aprendizado de máquina, aprendizado profundo, aprendizado por reforço
* Algoritmos de IA: árvores de decisão, redes neurais, máquinas de vetores de suporte
* Dados e recursos para IA

**Aplicações da IA:**

* Visão computacional (reconhecimento de imagem, processamento de vídeo)
* Processamento de linguagem natural (tradução, geração de texto)
* Robótica e automação
* Cuidados de saúde e medicina
* Finanças e negócios

**Desafios e Considerações Éticas:**

* Viés e discriminação em IA
* Privacidade e segurança de dados
* Impacto no emprego e na sociedade
* Regulamentações e políticas éticas para IA

**Aprendizado de Máquina:**

* Conceitos de aprendizado supervisionado, não supervisionado e por reforço
* Modelos de aprendizado de máquina: regressão, classificação, agrupamento
* Avaliação e otimização de modelos
* Técnicas de pré-processamento e engenharia de recursos

**Aprendizado Profundo:**

* Re

Gerando o chat boot

In [ ]:
chat = model.start_chat(history=[])

In [ ]:
prompt = input("Esperando o prompt: ")

while prompt != "fim":
  response = chat.send_message(prompt)
  print("Resposta: ", response.text, "\n")
  prompt = input("Esperando o prompt: ")

Esperando o prompt: Meu primo nasceu nessa cidade. Qual é a nacionalidade dele?
Resposta:  Japonesa 

Esperando o prompt: Qual é a população dessa cidade?
Resposta:  12,6 milhões (2023) 

Esperando o prompt: População metropolitana
Resposta:  37,34 milhões (2023) 

Esperando o prompt: fim


In [ ]:
# Melhorando a visualização
# Código disponível em https://ai.google.dev/tutorials/python_quickstart#import_packages

import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('``', '"')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Imprimindo o historico
for message in chat.history:
  display(to_markdown(f"**{message.role}**: {message.parts[0].text}"))
  print('-----------------------------------------')

# MQTT e Max6675

Instalando bibliotecas

In [ ]:
!pip install paho-mqtt
!pip install --upgrade paho-mqtt

Código da leitura

In [ ]:
# Instalação das bibliotecas:
# pip install paho-mqtt pandas

import paho.mqtt.client as mqtt
import pandas as pd
import time
from datetime import datetime

# Define o endereço do broker MQTT
mqttBroker = "broker.hivemq.com"
mqttPort = 1883
topic = "tacho/temperatura"

# Variáveis globais para armazenar os dados
temperaturas = []
tempos = []

# Função chamada quando a conexão com o broker é estabelecida
def on_connect(client, userdata, flags, rc):
  print("Conectado ao broker MQTT com código de resultado: "+str(rc))
  client.subscribe(topic)

# Função chamada quando uma mensagem é recebida
def on_message(client, userdata, message):
  global temperaturas, tempos
  temperatura = float(message.payload.decode("utf-8"))
  timestamp = datetime.now()
  temperaturas.append(temperatura)
  tempos.append(timestamp)
  print(f"[{timestamp}] Temperatura: {temperatura} °C")

# Função para salvar os dados em um arquivo
def salvar_dados():
  global temperaturas, tempos
  nome_arquivo = input("Digite o nome do arquivo com a extensão desejada (ex: dados.csv, dados.txt): ")
  df = pd.DataFrame({"Tempo": tempos, "Temperatura": temperaturas})

  if nome_arquivo.endswith(".csv"):
    df.to_csv(nome_arquivo, index=False)
  elif nome_arquivo.endswith(".txt"):
    with open(nome_arquivo, "w") as f:
      for tempo, temperatura in zip(tempos, temperaturas):
        f.write(f"{tempo},{temperatura}\n")
  else:
    print("Formato de arquivo não suportado. Use .csv ou .txt.")
    return

  print(f"Dados salvos em {nome_arquivo}")

# Função para retornar a temperatura a 'minutos' minutos atrás
def temperatura_minutos_atras(minutos):
  global temperaturas, tempos
  if len(tempos) == 0:
    return "Nenhum dado de temperatura disponível."
  tempo_alvo = datetime.now() - pd.Timedelta(minutes=minutos)
  for i in range(len(tempos) - 1, -1, -1):
    if tempos[i] <= tempo_alvo:
      return temperaturas[i]
  return "Nenhum dado encontrado para esse período."

# Função para retornar a temperatura a 'horas' horas atrás
def temperatura_horas_atras(horas):
  global temperaturas, tempos
  if len(tempos) == 0:
    return "Nenhum dado de temperatura disponível."
  tempo_alvo = datetime.now() - pd.Timedelta(hours=horas)
  for i in range(len(tempos) - 1, -1, -1):
    if tempos[i] <= tempo_alvo:
      return temperaturas[i]
  return "Nenhum dado encontrado para esse período."

# Função para retornar a temperatura máxima
def temperatura_maxima():
  global temperaturas
  if len(temperaturas) == 0:
    return "Nenhum dado de temperatura disponível."
  return max(temperaturas)

# Função para retornar a temperatura mínima
def temperatura_minima():
  global temperaturas
  if len(temperaturas) == 0:
    return "Nenhum dado de temperatura disponível."
  return min(temperaturas)

# Função principal do chatbot
def chatbot():
  while True:
    comando = input("Você deseja visualizar qual dado? (temperatura, max, min, salvar, sair): ")

    if comando == "temperatura":
      periodo = input("Digite o período de tempo (minutos ou horas): ")
      if periodo == "minutos":
        minutos = int(input("Digite o número de minutos: "))
        temperatura = temperatura_minutos_atras(minutos)
        print(f"Temperatura {minutos} minutos atrás: {temperatura} °C")
      elif periodo == "horas":
        horas = int(input("Digite o número de horas: "))
        temperatura = temperatura_horas_atras(horas)
        print(f"Temperatura {horas} horas atrás: {temperatura} °C")
      else:
        print("Período inválido. Use 'minutos' ou 'horas'.")

    elif comando == "max":
      temperatura = temperatura_maxima()
      print(f"Temperatura máxima: {temperatura} °C")

    elif comando == "min":
      temperatura = temperatura_minima()
      print(f"Temperatura mínima: {temperatura} °C")

    elif comando == "salvar":
      salvar_dados()

    elif comando == "sair":
      break

    else:
      print("Comando inválido.")

# Cria o cliente MQTT
client = mqtt.Client()

# Define as funções de callback
client.on_connect = on_connect
client.on_message = on_message

# Conecta ao broker MQTT
client.connect(mqttBroker, mqttPort)

# Inicia o loop de eventos em uma thread separada
client.loop_start()

# Inicia o chatbot
chatbot()

# Desconecta do broker MQTT
client.loop_stop()
client.disconnect()